In [31]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'tensorflow'

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# Configura la longitud de la secuencia y el horizonte de predicción
sequence_length = 24  #  24 pasos (2h 24min)
horizon = 1           # Predicción 1 paso adelante

# Función para crear ventanas de tiempo
def create_dataset(X, y, sequence_length=24, horizon=1):
    Xs, ys = [], []
    for i in range(len(X) - sequence_length - horizon + 1):
        Xs.append(X[i:(i + sequence_length)])
        ys.append(y[i + sequence_length + horizon - 1])
    return np.array(Xs), np.array(ys)

In [25]:


current_dir = os.getcwd()

X_train_path = os.path.join('..', 'data', 'processed', 'X_train.csv')
X_test_path = os.path.join('..', 'data', 'processed', 'X_test.csv')
y_train_path = os.path.join('..', 'data', 'processed', 'y_train.csv')
y_test_path = os.path.join('..', 'data', 'processed', 'y_test.csv')


In [26]:
print(X_test_path)
print(os.path.exists(X_test_path))

..\data\processed\X_test.csv
True


In [27]:

X_train = pd.read_csv(X_train_path)
X_test = pd.read_csv(X_test_path)
y_train = pd.read_csv(y_train_path)
y_test = pd.read_csv(y_test_path)

Despues de cargados los dataframes vamos a construir el modelo con Keras, trataremos de predecir en simultaneo la temperatura y precipitación usando una RNN

Primero nos aseguraremos que las dimensiones de los dataframes sean las adecuadas.

N_features hace referencia a la cantidad de variables categoricas que miden en cada paso con el paso del tiempo. que en nuestro caso será 9 pues son viene dado por las columnas : dd

ff

hu

td

month

quarter

dd_rad

wind_u

wind_v


In [29]:
N_features = X_train.shape[1]  # 9 características (excluyendo 'date')
N_timesteps = X_train.shape[1]  # Usaremos la misma cantidad de pasos de tiempo que características (esto depende de cómo formatees tus datos)
N_output = 2  # Para predecir temperatura (t) y precipitación (precip)

Usando LSTM

In [30]:
# Crear el modelo
model = models.Sequential()

NameError: name 'models' is not defined

In [ ]:
# Capa LSTM
model.add(layers.LSTM(units=64, return_sequences=False, input_shape=(N_timesteps, N_features)))

# Dropout para evitar sobreajuste
model.add(layers.Dropout(0.2))

# Capa densa para la predicción
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.2))

# Capa de salida
model.add(layers.Dense(N_output))  # Dos salidas: temperatura (t) y precipitación (precip)


In [ ]:
# Compilación del modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Ver resumen del modelo
model.summary()

# Definir callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [ ]:
# Entrenamiento del modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


In [ ]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}")
print(f"Test MAE: {test_mae}")

Usando GRU

In [ ]:
model_gru = models.Sequential()

In [ ]:
# Capa GRU
model_gru.add(layers.GRU(units=64, return_sequences=False, input_shape=(N_timesteps, N_features)))

# Dropout para evitar sobreajuste
model_gru.add(layers.Dropout(0.2))

# Capa densa
model_gru.add(layers.Dense(64, activation='relu'))
model_gru.add(layers.Dropout(0.2))

# Capa de salida
model_gru.add(layers.Dense(N_output))


In [ ]:
model_gru.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


In [ ]:
model_gru.summary()

In [ ]:
history_gru = model_gru.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


In [ ]:
test_loss_gru, test_mae_gru = model_gru.evaluate(X_test, y_test)
print(f"GRU Test loss: {test_loss_gru}")
print(f"GRU Test MAE: {test_mae_gru}")

Haremos una comparación visual mediante graficos de entrenamiento y validacion para ver como se comporta la convergencia de los modelos y tambien su peridda

In [ ]:
# Graficar las pérdidas de entrenamiento y validación para LSTM y GRU
plt.plot(history.history['loss'], label='LSTM Train Loss')
plt.plot(history.history['val_loss'], label='LSTM Validation Loss')
plt.plot(history_gru.history['loss'], label='GRU Train Loss')
plt.plot(history_gru.history['val_loss'], label='GRU Validation Loss')
plt.title('Train vs Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Graficar los MAE de LSTM y GRU
plt.plot(history.history['mae'], label='LSTM Train MAE')
plt.plot(history.history['val_mae'], label='LSTM Validation MAE')
plt.plot(history_gru.history['mae'], label='GRU Train MAE')
plt.plot(history_gru.history['val_mae'], label='GRU Validation MAE')
plt.title('Train vs Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()